Let's create an InfluxDB client by importing the modules and setting all the required configuration. The access token is taken from ENV, be sure to keep it in `.env` around before you start the jupyter notebook server.

In [ ]:
import os

import pandas as pd
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

# configure influxdb client
BUCKET = "racing"
ORG = "b4mad"
TOKEN = os.environ.get("INFLUXDB_TOKEN")
URL = "https://telemetry.b4mad.racing/"

# and create the client and a quary api
client = influxdb_client.InfluxDBClient(
   url=URL,
   token=TOKEN,
   org=ORG
)

query_api = client.query_api()

In [ ]:
query = f"""
from(bucket: "{BUCKET}")
  |> range(start: -30d, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "laps")
  |> filter(fn: (r) => r["_field"] == "TrackPositionPercent" or r["_field"] == "Brake" or r["_field"] == "CurrentLap")
  |> filter(fn: (r) => r["SessionId"] == "1665367439")
  |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> sort(columns: ["_time"], desc: false)
  |> keep(columns: ["_time", "CarModel", "TrackCode", "Brake", "TrackPositionPercent", "CurrentLap"])
  |> fill(usePrevious: true, column: "CurrentLap")
  |> fill(usePrevious: true, column: "Brake")
"""

df = query_api.query_data_frame(org=ORG, query=query)

In [ ]:
for c in df.columns:
    print(c, len(df[c].unique()))

In [ ]:
df['datetime'] = pd.to_datetime(df['_time'])
df.set_index('datetime',inplace=True)
df.sort_index(inplace=True)
plt.rcParams["figure.figsize"] = (10,2)
df.plot()

In [ ]:
#df= df.pivot(index='TrackPositionPercent',columns=['CurrentLap', 'Brake'], values=['CurrentLap', 'Brake'])
#df= df.pivot(index='TrackPositionPercent',columns=['CurrentLap', 'Brake']) 
df


In [ ]:
#df['Gear'].replace('N',0,inplace=True)

#df.index = pd.to_datetime(df.index)
#df.sort_index(inplace=True)
df

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,20)
cols=['Brake', 'TrackPositionPercent', 'CurrentLap']
fig, ax = plt.subplots(len(cols))
for i, c in enumerate(cols):
    df[c].astype(float).plot(ax=ax[i])
    ax[i].set_title(c)
plt.show()


In [ ]:
lap = 3
df[df['CurrentLap']==lap]
plt.rcParams["figure.figsize"] = (20,20)
fig, ax = plt.subplots(len(cols))
for i, c in enumerate(cols):
    df.loc[df['CurrentLap']==lap,c].astype(float).plot(ax=ax[i])
    ax[i].set_title(c)
plt.show()


In [ ]:
df[df['CurrentLap']==lap]


In [ ]:
import seaborn as sns

plt.rcParams["figure.figsize"] = (20,5)
for c in ['Brake']:
    sns.lineplot(data=df,x='TrackPositionPercent',y=c,hue='CurrentLap', legend=False)
    plt.show()